# UCI MTB DH Data Retrieval

## Setup
#### Import Libraries

If you do not have these libraries available, you should install them using `pip`

```
pip install requests
pip install bs4
pip install pandas
```

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import os

In [2]:
def calculate_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

Widen display area to prevent column wrapping, and always show all columns for debug 

In [3]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

## Config

Which race data are we collecting?

1. Losinj
1. Fort William
1. Leogang
1. Val di Sole
1. Vallnord
1. Mont-Sainte-Anne
1. La Bresse

In [4]:
race = 3
gender = 'm'
event = 'dh'
rnrSpeed = False

#### Data Sources

The UCI Live Timing API contains a lot of data points, but not all the ones we want (speed being the main one missing), and not even all the ones they include on their own PDF which is frustrating.

Similarly, Roots & Rain also has a lot of the data points, but again not all of them; most notably it's missing timing splits 4 and 5.

Therefore we need to pull from both sources and combine the sets.

We specify the URLs for both sources from which we will extract our data. The UCI API URL can be found by loading the Live Timing page then using your browser's inspector on the Network tab (in Chrome at least) to see the data feed. As the UCI seems to be using a Single Page Application (SPA) here, it's not straight forward to extract this link automagically.

**Note:** The Race list is now maintained as an external python config file `config.py` imported below

In [5]:
from config import races

racename = races[race]['name']
urlUci = races[race]['urls']['uci'] + str(( 3 if 'm' == gender else 6 )) + '/'
urlUciQ = races[race]['urls']['uci'] + str(( 2 if 'm' == gender else 5 )) + '/'
urlRoots = races[race]['urls']['rnr'] + gender + '/'

File handling setup

In [6]:
directory = event + str(race) + '_' + racename
if not os.path.exists(directory):
    os.makedirs(directory)

file_prefix = event + str(race) + '_' + racename + '_' + gender
file_prefix = os.path.join( directory, file_prefix )

# UCI API
### Load Data

These two lines make the actual request to the server, and then converts the JSON string response in to a usable list format (deserialization)

In [7]:
r = requests.get( urlUci ).json()
q = requests.get( urlUciQ ).json()

The API returns with three main sections:

1. `Last Finisher`
 - Racers in order of start time
2. `Results`
 - Racers in finishing rank order
3. `Riders`
 - Personal details on all racers
 
Each contains many data points. To see all the contained data, you can un-comment and execute any of the lines in the next section to explore more.

In [8]:
# display( q )
# display( d['Results'][7] )
# display( d['Riders']['1001'] )
# display( d['Results'][61] )

### Extract Data

Here we iterate over the `Results` sub-set of data to extract the information we care about: basically some identifying info, and their splits.

There is a loop within a loop here as the first iterates over the two result sets qualifying and race, and within that we extract the necessary stats.

If you looked at detail of the returned data set in the last step you might have noticed the rider's name is not stored next to their result, riders are only identified by a reference number. To facilitate our analysis later on it is useful to import each rider's name at this stage by cross-referencing the `Riders` sub-set.

We start with an empty list `lst` and in each loop iteration add an entry (actually a dict) to that list for each rider.

In [9]:
dat = {}
for i, d in enumerate( [ r, q ] ):
    lastStart = d['Riders'][list(d['Riders'].keys())[-1]]['StartOrder']
    pfx = 'q_' if 1 == i else ''

    splits = len(d['Results'][0]['Times'] )
    lst = []
    for idx, row in enumerate( d['Results'] ):
        fin = "Finished" == row['Status']
        res = {
            'name': d['Riders'][str(row['RaceNr'])]['PrintName'],
            'id': row['RaceNr'],
            'uci': d['Riders'][str(row['RaceNr'])]['UciRiderId'],
            'bib': d['Riders'][str(row['RaceNr'])]['RaceNr'],
            pfx + 'status': row['Status'],
            pfx + 'rank': row['Position'] if fin else idx+1,
            pfx + 'start': d['Riders'][str(row['RaceNr'])]['StartOrder'],
            pfx + 'start_rev': lastStart - d['Riders'][str(row['RaceNr'])]['StartOrder'] +1
        }
        if rnrSpeed:
            res[pfx + 'speed'] = np.nan

        # Add all splits to result set
        for split in range( 0, splits ):
            head = pfx + 'split'
            res[head + str(split+1)] = row['Times'][split]['RaceTime']/1000 if fin else np.nan

        # Append result set to list
        lst.append(res)

    dat[i] = lst

Here we load the completed lists in to Pandas dataframes to facilitate working with the data moving forward

In [10]:
df = pd.DataFrame( dat[0] )
dq = pd.DataFrame( dat[1] )

# Points

Neither data set contains points awarded so we use a reference file and merge

Merge type here must be `outer` so people that finished outside the top 60 men, top 15 women, or DNF, don't get trimmed from the dataset

In [11]:
df_qp = pd.read_csv( event + '_points_qual_' + gender + '.csv', index_col=0 )
df_qp = df_qp.reset_index(drop=False)
dq = dq.merge( df_qp, left_index=True, right_index=True, how="outer")

df_rp = pd.read_csv( event + '_points_race_' + gender + '.csv', index_col=0 )
df_rp = df_rp.reset_index(drop=False)
df = df.merge( df_rp, left_index=True, right_index=True, how="outer")

In [12]:
display( df.head(), dq.head() )

,bib,id,name,rank,split1,split2,split3,split4,split5,start,start_rev,status,uci,r_points
0,3,1003,PIERRON Amaury,1,45.256,80.744,118.416,160.917,190.991,60,3,Finished,10008827283,200.0
1,1,1001,GWIN Aaron,2,45.525,80.466,118.357,161.112,191.499,52,11,Finished,10006516663,160.0
2,10,1010,GREENLAND Laurie,3,45.649,81.097,119.363,161.821,192.146,57,6,Finished,10009404738,140.0
3,4,1004,BROSNAN Troy,4,45.808,81.241,119.218,161.996,192.305,56,7,Finished,10007307417,125.0
4,2,1002,VERGIER Loris,5,46.139,81.510,119.483,162.984,193.121,58,5,Finished,10008723112,110.0


,bib,id,name,q_rank,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points
0,6,1006,SHAW Luca,1,47.311,83.825,122.298,165.051,195.890,6,153,Finished,10008813442,50.0
1,12,1012,MACDONALD Brook,2,46.528,83.026,121.850,165.415,196.036,12,147,Finished,10006429969,40.0
2,3,1003,PIERRON Amaury,3,47.141,83.873,122.846,165.982,196.528,3,156,Finished,10008827283,30.0
3,17,1017,BRUNI Loic,4,46.232,82.965,122.055,165.441,196.647,17,142,Finished,10007544358,25.0
4,2,1002,VERGIER Loris,5,47.097,83.236,122.127,166.501,197.230,2,157,Finished,10008723112,22.0


# Merge and Expand

This code merges the qualifying and race data in to a single data frame, merging only the columns that are unique between them to avoid duplicates. This allows us to do more in depth analysis later on.

As we merged race in to quali, we re-sort the resulting dataset by race rank

In [13]:
dfq = dq.merge( df[['id'] + list(df.columns.difference( dq.columns ))], left_on='id', right_on='id', how='outer' )
dfq = dfq.sort_values( 'rank', ascending=True )
dfq = dfq.reset_index( drop=True )
dfq['points'] = dfq['r_points'].fillna(0) + dfq['q_points'].fillna(0)

# Time difference between race and quali
dfq['qr_diff'] = dfq['split5'] - dfq['q_split5']

#### Expand Dataset

Calculate and add all the extra columns we need for split and sector differences and their rankings

In [14]:
for pfx in [ 'q_', '' ]:
    for i in range( 1, splits+1 ):
        split = pfx + 'split' + str(i)
        sector = split + '_sector'
        dfq[split + '_rank'] = dfq[split].rank(method='dense')
        dfq[split + '_vs_best'] = (dfq[split] - dfq[split].min())
        dfq[split + '_vs_winner'] = (dfq[split] - dfq[split][0])

        if i > 1:
            dfq[split + '_sector'] = dfq[split] - dfq[pfx + 'split' + str(i-1)]
            dfq[split + '_sector_rank'] = dfq[sector].rank(method='dense')
            dfq[split + '_sector_vs_best'] = (dfq[sector] - dfq[sector].min())
            dfq[split + '_sector_vs_winner'] = (dfq[sector] - dfq[sector][0])

We can take a peek at our data at this point to make sure it looks how we expect.

At this point the `speed` column is NaN (Not a Number) for all racers. This will be filled in below.

In [15]:
display( dfq.head(10) )

,bib,id,name,q_rank,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points,r_points,rank,split1,split2,split3,split4,split5,start,start_rev,status,points,qr_diff,q_split1_rank,q_split1_vs_best,q_split1_vs_winner,q_split2_rank,q_split2_vs_best,q_split2_vs_winner,q_split2_sector,q_split2_sector_rank,q_split2_sector_vs_best,q_split2_sector_vs_winner,q_split3_rank,q_split3_vs_best,q_split3_vs_winner,q_split3_sector,q_split3_sector_rank,q_split3_sector_vs_best,q_split3_sector_vs_winner,q_split4_rank,q_split4_vs_best,q_split4_vs_winner,q_split4_sector,q_split4_sector_rank,q_split4_sector_vs_best,q_split4_sector_vs_winner,q_split5_rank,q_split5_vs_best,q_split5_vs_winner,q_split5_sector,q_split5_sector_rank,q_split5_sector_vs_best,q_split5_sector_vs_winner,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner
0,3,1003,PIERRON Amaury,3,47.141,83.873,122.846,165.982,196.528,3,156,Finished,10008827283,30.0,200.0,1.0,45.256,80.744,118.416,160.917,190.991,60.0,3.0,Finished,230.0,-5.537,11.0,1.123,0.000,10.0,0.908,0.000,36.732,6.0,0.593,0.000,7.0,0.996,0.000,38.973,7.0,0.500,0.000,4.0,0.931,0.000,43.136,2.0,0.383,0.000,3.0,0.638,0.000,30.546,2.0,0.106,0.000,1.0,0.000,0.000,2.0,0.278,0.000,35.488,5.0,0.547,0.000,3.0,0.059,0.000,37.672,3.0,0.461,0.000,1.0,0.000,0.000,42.501,2.0,0.043,0.000,1.0,0.000,0.000,30.074,2.0,0.046,0.000
1,1,1001,GWIN Aaron,13,48.361,85.469,124.547,169.081,200.424,1,158,Finished,10006516663,12.0,160.0,2.0,45.525,80.466,118.357,161.112,191.499,52.0,11.0,Finished,172.0,-8.925,41.0,2.343,1.220,22.0,2.504,1.596,37.108,16.0,0.969,0.376,12.0,2.697,1.701,39.078,8.0,0.605,0.105,13.0,4.030,3.099,44.534,20.0,1.781,1.398,13.0,4.534,3.896,31.343,20.0,0.903,0.797,3.0,0.269,0.269,1.0,0.000,-0.278,34.941,1.0,0.000,-0.547,1.0,0.000,-0.059,37.891,5.0,0.680,0.219,2.0,0.195,0.195,42.755,4.0,0.297,0.254,2.0,0.508,0.508,30.387,9.0,0.359,0.313
2,10,1010,GREENLAND Laurie,6,46.532,83.007,122.780,166.467,197.595,10,149,Finished,10009404738,20.0,140.0,3.0,45.649,81.097,119.363,161.821,192.146,57.0,6.0,Finished,160.0,-5.449,5.0,0.514,-0.609,2.0,0.042,-0.866,36.475,2.0,0.336,-0.257,6.0,0.930,-0.066,39.773,30.0,1.300,0.800,5.0,1.416,0.485,43.687,5.0,0.934,0.551,6.0,1.705,1.067,31.128,11.0,0.688,0.582,6.0,0.393,0.393,3.0,0.631,0.353,35.448,4.0,0.507,-0.040,5.0,1.006,0.947,38.266,12.0,1.055,0.594,3.0,0.904,0.904,42.458,1.0,0.000,-0.043,3.0,1.155,1.155,30.325,8.0,0.297,0.251
3,4,1004,BROSNAN Troy,7,47.002,83.781,122.684,166.936,197.748,4,155,Finished,10007307417,18.0,125.0,4.0,45.808,81.241,119.218,161.996,192.305,56.0,7.0,Finished,143.0,-5.443,8.0,0.984,-0.139,7.0,0.816,-0.092,36.779,10.0,0.640,0.047,5.0,0.834,-0.162,38.903,6.0,0.430,-0.070,7.0,1.885,0.954,44.252,12.0,1.499,1.116,7.0,1.858,1.220,30.812,5.0,0.372,0.266,9.0,0.552,0.552,5.0,0.775,0.497,35.433,3.0,0.492,-0.055,4.0,0.861,0.802,37.977,7.0,0.766,0.305,4.0,1.079,1.079,42.778,5.0,0.320,0.277,4.0,1.314,1.314,30.309,7.0,0.281,0.235
4,2,1002,VERGIER Loris,5,47.097,83.236,122.127,166.501,197.230,2,157,Finished,10008723112,22.0,110.0,5.0,46.139,81.510,119.483,162.984,193.121,58.0,5.0,Finished,132.0,-4.109,10.0,1.079,-0.044,5.0,0.271,-0.637,36.139,1.0,0.000,-0.593,3.0,0.277,-0.719,38.891,5.0,0.418,-0.082,6.0,1.450,0.519,44.374,15.0,1.621,1.238,5.0,1.340,0.702,30.729,4.0,0.289,0.183,12.0,0.883,0.883,6.0,1.044,0.766,35.371,2.0,0.430,-0.117,6.0,1.126,1.067,37.973,6.0,0.762,0.301,5.0,2.067,2.067,43.501,14.0,1.043,1.000,5.0,2.130,2.130,30.137,3.0,0.109,0.063
5,9,1009,NORTON Dakotah,57,47.522,84.61

#### Rider Data

Saving the personal information about each racer is much easier as we can just export the entire `Riders` dataset. However, the rows and columns are the wrong way round so the `.T` command *transposes* the information, meaning it basically flips the axes.

In [16]:
df2 = pd.DataFrame( d['Riders'] )
df2 = df2.T
df2['Age'] = [ calculate_age( dt.datetime.strptime( dob[:10], "%Y-%m-%d" ) ) for dob in df2['BirthDate'] ]

Here we can glimpse the first few rows of our `DataFrame` and can check the data looks as we expect

In [17]:
display( df2.head() )

,BirthDate,CategoryCode,FamilyName,GivenName,Id,Nation,Outfit,PrintName,RaceId,RaceNr,ScoreboardName,StartOrder,StartTime,UciCode,UciRank,UciRiderId,UciTeamCode,UciTeamId,UciTeamName,WorldCupRank,Age
1001,1987-12-24T00:00:00,ME,GWIN,Aaron,1197024565271553,USA,WCL,GWIN Aaron,0,1,GWIN A,1,50400000,USA19871224,1,10006516663,YTM,1531,THE YT MOB,1,30
1002,1996-05-07T00:00:00,ME,VERGIER,Loris,1197024565271554,FRA,NaN,VERGIER Loris,0,2,VERGIER L,2,50430000,FRA19960507,5,10008723112,SCB,1307,SANTA CRUZ SYNDICATE,2,22
1003,1996-03-04T00:00:00,ME,PIERRON,Amaury,1197024565271555,FRA,NaN,PIERRON Amaury,0,3,PIERRON A,3,50460000,FRA19960304,6,10008827283,CVN,1590,COMMENCAL / VALLNORD,3,22
1004,1993-07-13T00:00:00,ME,BROSNAN,Troy,1197024565271556,AUS,NCh,BROSNAN Troy,0,4,BROSNAN T,4,50490000,AUS19930713,2,10007307417,CFT,2162,CANYON FACTORY DOWNHILL TEAM,4,24
1005,1988-10-28T00:00:00,ME,BLENKINSOP,Samuel,1197024565271557,NZL,CCh,BLENKINSOP Samuel,0,5,BLENKINSOP S,5,50520000,NZL19881028,7,10004485929,NFR,2013,NORCO FACTORY RACING,5,29


# Speed Data

Roots and Rain seem to take about 3 days to get their results online. Given all UCI data is available immediately I have added a second method for getting speed data. There is boolean in the config at top of this notebook to decide if we pull data from RnR or we use an import CSV file.

## Roots and Rain

### Load Data

Similar to the UCI api, we make a request to the server with the previously declared `urlRoots` variable. This time however we simply load the content of the response as text which is actually the HTML code of the web page. We do not do have a nice JSON API to read which means we will not deserialize.

Next we invoke a utility called `BeautifulSoup` to help us extract the data from this messy HTML code

In [18]:
if rnrSpeed:
    c = requests.post( urlRoots ).content
    soup = BeautifulSoup( c, "html.parser" )

### Extract Data

If you look at the Roots and Rain page you'll see it listed in a tabular format. What we do here is find all the rows of that table so we can extract the information we need.

Specifically we are looking for instances of `tr` (table row), with a class that *begins with* `c-` as this is a common denomenator I discovered when looking through the code with the browser inspector

In [19]:
if rnrSpeed:
    rows = soup.find_all( "tr", class_=lambda x: x and 'c-' in x )

Similar to the UCI data set, here we will iterate over each row in our data set--basically each table row from the web page--and extract the bits we need.

Racer speed is the metric we're interested in, but in order to match that to our existing data set we need a corresponding identifier so we also extract the racer licence number as that exists in both sets and we can match them together: it is the *intersect* between both sets of data.

To summarise:
1. Extract licence number and corresponding speed
2. Import speed to existing DataFrame matching racers by licence

The `if` condition in the middle will exit this block of code once we hit the end of the Elite finishers, seeing as that's all we have in our existing data set so can't match anyone else

In [20]:
if rnrSpeed:
    for row in rows:
        cells = row.find_all( "td" )
        qspd = cells[7].text[:5]
        spd = cells[12].text[:5]
        qspeed = float( qspd if 0 < len(qspd) else 0 )
        speed = float( spd if 0 < len(spd) else 0 )
        licence = cells[4].text
        bib = int( cells[1].text )
        pos = cells[0].text[8:]
        if "" == pos: break

        # Match rider by UCI licence if present, otherwise fallback to bib
        if len(df2.loc[df2['UciRiderId'] == licence].index.values ):
            rid = int(df2.loc[df2['UciRiderId'] == licence].index.values[0])
        else:
            rid = int( df2.loc[df2['RaceNr'] == bib].index.values[0] )

        # Add speed, and other associated metrics
        dfq.loc[dfq['id'] == rid, 'speed'] = speed
        dfq.loc[dfq['id'] == rid, 'q_speed'] = qspeed

As before, we can take another look at how our data is looking, with the `speed` column now containing data 

## UCI PDF Converted Speed

Despite UCI having a speed field in the splits data of their API, it is always 0. Thanks. They do make that data available in their PDFs, but that data is not easy to extract and all regular converters fail. However, trying with some OCR engines I did have good success. The best of which is https://convertio.co/ocr/. I take the converted file, strip it down to UCI# and speed, save as CSV, and then import and merge here.

Regex code for removing (X) rank from OCR converted files.

> Find: `(,[0-9\.]+).*`
>
> Replace: `$1`

In [21]:
dfs = pd.read_csv( file_prefix + '.speeds.csv' )
dfsq = pd.read_csv( file_prefix + '.qspeeds.csv' )
dfq.uci = dfq.uci.astype(str)
dfs.uci = dfs.uci.astype(str)
dfsq.uci = dfsq.uci.astype(str)

dfq = dfq.merge( dfs, left_on='uci', right_on='uci', how='left' )
dfq = dfq.merge( dfsq, left_on='uci', right_on='uci', how='left' )
# dfqs[['name', 'uci', 'q_speed', 'speed']]
# dfqs.columns

In [22]:
display( dfq.head() )

,bib,id,name,q_rank,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points,r_points,rank,split1,split2,split3,split4,split5,start,start_rev,status,points,qr_diff,q_split1_rank,q_split1_vs_best,q_split1_vs_winner,q_split2_rank,q_split2_vs_best,q_split2_vs_winner,q_split2_sector,q_split2_sector_rank,q_split2_sector_vs_best,q_split2_sector_vs_winner,q_split3_rank,q_split3_vs_best,q_split3_vs_winner,q_split3_sector,q_split3_sector_rank,q_split3_sector_vs_best,q_split3_sector_vs_winner,q_split4_rank,q_split4_vs_best,q_split4_vs_winner,q_split4_sector,q_split4_sector_rank,q_split4_sector_vs_best,q_split4_sector_vs_winner,q_split5_rank,q_split5_vs_best,q_split5_vs_winner,q_split5_sector,q_split5_sector_rank,q_split5_sector_vs_best,q_split5_sector_vs_winner,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner,speed,q_speed
0,3,1003,PIERRON Amaury,3,47.141,83.873,122.846,165.982,196.528,3,156,Finished,10008827283,30.0,200.0,1.0,45.256,80.744,118.416,160.917,190.991,60.0,3.0,Finished,230.0,-5.537,11.0,1.123,0.000,10.0,0.908,0.000,36.732,6.0,0.593,0.000,7.0,0.996,0.000,38.973,7.0,0.500,0.000,4.0,0.931,0.000,43.136,2.0,0.383,0.000,3.0,0.638,0.000,30.546,2.0,0.106,0.000,1.0,0.000,0.000,2.0,0.278,0.000,35.488,5.0,0.547,0.000,3.0,0.059,0.000,37.672,3.0,0.461,0.000,1.0,0.000,0.000,42.501,2.0,0.043,0.000,1.0,0.000,0.000,30.074,2.0,0.046,0.000,57.643,57.157
1,1,1001,GWIN Aaron,13,48.361,85.469,124.547,169.081,200.424,1,158,Finished,10006516663,12.0,160.0,2.0,45.525,80.466,118.357,161.112,191.499,52.0,11.0,Finished,172.0,-8.925,41.0,2.343,1.220,22.0,2.504,1.596,37.108,16.0,0.969,0.376,12.0,2.697,1.701,39.078,8.0,0.605,0.105,13.0,4.030,3.099,44.534,20.0,1.781,1.398,13.0,4.534,3.896,31.343,20.0,0.903,0.797,3.0,0.269,0.269,1.0,0.000,-0.278,34.941,1.0,0.000,-0.547,1.0,0.000,-0.059,37.891,5.0,0.680,0.219,2.0,0.195,0.195,42.755,4.0,0.297,0.254,2.0,0.508,0.508,30.387,9.0,0.359,0.313,58.566,57.715
2,10,1010,GREENLAND Laurie,6,46.532,83.007,122.780,166.467,197.595,10,149,Finished,10009404738,20.0,140.0,3.0,45.649,81.097,119.363,161.821,192.146,57.0,6.0,Finished,160.0,-5.449,5.0,0.514,-0.609,2.0,0.042,-0.866,36.475,2.0,0.336,-0.257,6.0,0.930,-0.066,39.773,30.0,1.300,0.800,5.0,1.416,0.485,43.687,5.0,0.934,0.551,6.0,1.705,1.067,31.128,11.0,0.688,0.582,6.0,0.393,0.393,3.0,0.631,0.353,35.448,4.0,0.507,-0.040,5.0,1.006,0.947,38.266,12.0,1.055,0.594,3.0,0.904,0.904,42.458,1.0,0.000,-0.043,3.0,1.155,1.155,30.325,8.0,0.297,0.251,57.759,55.398
3,4,1004,BROSNAN Troy,7,47.002,83.781,122.684,166.936,197.748,4,155,Finished,10007307417,18.0,125.0,4.0,45.808,81.241,119.218,161.996,192.305,56.0,7.0,Finished,143.0,-5.443,8.0,0.984,-0.139,7.0,0.816,-0.092,36.779,10.0,0.640,0.047,5.0,0.834,-0.162,38.903,6.0,0.430,-0.070,7.0,1.885,0.954,44.252,12.0,1.499,1.116,7.0,1.858,1.220,30.812,5.0,0.372,0.266,9.0,0.552,0.552,5.0,0.775,0.497,35.433,3.0,0.492,-0.055,4.0,0.861,0.802,37.977,7.0,0.766,0.305,4.0,1.079,1.079,42.778,5.0,0.320,0.277,4.0,1.314,1.314,30.309,7.0,0.281,0.235,57.200,56.333
4,2,1002,VERGIER Loris,5,47.097,83.236,122.127,166.501,197.230,2,157,Finished,10008723112,22.0,110.0,5.0,46.139,81.510,119.483,162.984,193.121,58.0,5.0,Finished,132.0,-4.109,10.0,1.079,-0.044,5.0,0.271,-0.637,36.139,1.0,0.000,-0.593,3.0,0.277,-0.719,38.891,5.0,0.418,-0.082,6.0,1.450,0.519,44.374,15.0,1.621,1.238,5.0,1.340,0.702,30.729,4.0,0.289,0.183,12.0,0.883,0.883,6.0,1.044,0.766,35.371,2.0,0.430,-0.117,6.0,1.126,1.067,37.973,6.0,0.762,0.301,5.0,2.067,2.067,43.501,14.0,1.043,1.000,5.0,2.13

Now we have speed info either way, expand data set

In [27]:
dfq['speed_ms'] = dfq['speed'] * (1000/60/60)
dfq['speed_ms_vs_best'] = dfq['speed_ms'].max() - dfq.speed_ms
dfq['speed_rank'] = dfq.speed.rank(method='dense', ascending=False)
dfq['q_speed_rank'] = dfq.q_speed.rank(method='dense', ascending=False)

# Data Export

All that's left is to save our data to CSV files so we can quickly import it again for analysis and visualization without making constant requests to the online servers. This not only reduces load on the services providing the data, but also allows us to work on our analysis "offline", moreover giving us a local copy in case the results are ever taken down. It's also much quicker to load data this way than constantly hitting online servers.

In [28]:
dfq.id = dfq.id.astype(str)
dfm = dfq.merge( df2, left_on='id', right_index=True, how='inner' )

In [29]:
df.to_csv( file_prefix + '.results.csv' )
dq.to_csv( file_prefix + '.quali.csv' )
df2.to_csv( file_prefix + '.racers.csv' )
dfm.to_csv( file_prefix + '.merged.csv' )

--- 

## Credits

### Author: Dominic Wrapson


> **@domwrap**
<br>
<img src="https://png.icons8.com/material/24/000000/github-2.png">
<img src="https://png.icons8.com/material/24/000000/stackoverflow.png">
<img src="https://png.icons8.com/material/24/000000/linkedin.png">
<img src="https://png.icons8.com/material/24/000000/windows8.png">
<img src="https://png.icons8.com/ios-glyphs/24/000000/instagram-new.png">
<img src="https://png.icons8.com/material/24/000000/twitter.png">
<a href="https://medium.com/@domwrap"><img src="https://png.icons8.com/material/24/000000/medium-logo.png"></a>
>
> <img src="https://png.icons8.com/material/24/000000/home.png"> http://domwrap.me
>
><img src="https://png.icons8.com/material/24/000000/cycling-mountain-bike.png"> [Hwulex](https://www.pinkbike.com/u/Hwulex/)


---

#### Special Thanks

Mark Shilton for the inspiration
- http://lookatthestats.blogspot.ca
- https://plus.google.com/+MarkShilton
- https://dirtmountainbike.com/author/mrgeekstats


<a href="https://icons8.com">Icon pack by Icons8</a>